# OpenFermion Fermionic Quantum Emulator 
## Phase I Design Proposal

This document outlines the functionality of the API for the OpenFermion Fermionic Quantum Emulator as defined in the design proposal.

 The code is covered and tested with pytest-cov at 99%. Toggle the print option to see output.

In [24]:
fqeprint = True

### 2.A.2 API for data conversion between OpenFermion's backend and OpenFermion-FQE
The API supports data conversion between native Cirq wavefunctions and the FQE wavefunction for arbitrary particle number and spin configuration.  A linequbit wavefunction result is a normalized numpy array.

In [25]:
import fqe
import numpy
import cirq

nqubit = 4

cirq_wfn = fqe.util.rand_wfn(2**nqubit).astype(numpy.complex64)
wfn_from_cirq = fqe.from_cirq(cirq_wfn, 0.0001)

if fqeprint:
    print('Inital wavefunction is \n {} \n with norm {}'.format(cirq_wfn, numpy.vdot(cirq_wfn, cirq_wfn)))
    wfn_from_cirq.print_wfn()

Inital wavefunction is 
 [ 0.09645153+9.2140652e-02j  0.16145852-7.0114821e-02j
  0.08239555+3.7468529e-01j  0.31179786-1.2040638e-01j
 -0.14026734+2.8639746e-01j -0.18464154+2.2202735e-01j
  0.04735724-2.2589865e-01j  0.20174037+2.0686507e-01j
  0.00361368-5.3627882e-02j -0.06807666+1.6205184e-01j
  0.2513582 -2.5031602e-01j -0.15411828-1.7915983e-01j
 -0.01153528+8.9576833e-02j -0.14375174-3.4623015e-01j
 -0.04816512-2.3907801e-02j  0.05842811-1.0092586e-04j] 
 with norm (1+0j)

Configuration nelectrons: 0 m_s: 0
Vector : 0
 a'00'b'00' : (0.0964515283703804+0.09214065223932266j)

Configuration nelectrons: 1 m_s: -1
Vector : 0
 a'00'b'01' : (-0.14026734232902527+0.28639745712280273j)
 a'00'b'10' : (0.16145852208137512-0.07011482119560242j)

Configuration nelectrons: 1 m_s: 1
Vector : 0
 a'01'b'00' : (0.0036136810667812824-0.05362788215279579j)
 a'10'b'00' : (0.08239555358886719+0.37468528747558594j)

Configuration nelectrons: 2 m_s: -2
Vector : 0
 a'00'b'11' : (-0.1846415400505066+0.2

### 2.A.3 Initialization of wave functions
FQE wave functions can be initialized by calling the constructor directly. fqe.get_wavefunction and get_wavefunction_mutltiple are utility wrappers that provide wrappers for common initialization motifs. e.g. get_spin_nonconserving_wavefunction will build a wavefunction that permits spin breaking Operators to be applied while get_wavefunction_multiple will return a list of initialize wavefunctions

In [26]:
wfFqe = fqe.Wavefunction([[4, -2, 10]])
wfFqe = fqe.get_wavefunction(4, -2, 10)

In [27]:
n = 3
m = -2
wfFQE = fqe.Wavefunction([[n, m, 10], [n+1, m+2, 10]])
wfFQE = fqe.get_wavefunction_multiple([[4, 0, 10], [5, -5, 10]])

In [28]:
wfFQE = fqe.get_spin_nonconserving_wavefunction(2)
wfFQE.set_wfn(strategy='random')
if fqeprint:
    wfFQE.print_wfn()


Configuration nelectrons: 2 m_s: 0
Vector : 0
 a'0001'b'0001' : (-0.04786796122789383-0.11286675930023193j)
 a'0001'b'0010' : (-0.15004976093769073-0.0017647177446633577j)
 a'0001'b'0100' : (-0.08476771414279938-0.007398678921163082j)
 a'0001'b'1000' : (-0.2784791886806488-0.09361627697944641j)
 a'0010'b'0001' : (0.08812709897756577+0.36353716254234314j)
 a'0010'b'0010' : (-0.05687916278839111-0.32680338621139526j)
 a'0010'b'0100' : (0.12052047997713089-0.08402305096387863j)
 a'0010'b'1000' : (0.14810921251773834+0.04006848484277725j)
 a'0100'b'0001' : (-0.22645647823810577+0.05994859337806702j)
 a'0100'b'0010' : (0.21680548787117004-0.1661015748977661j)
 a'0100'b'0100' : (0.0844455286860466-0.26342910528182983j)
 a'0100'b'1000' : (-0.1589016169309616+0.013544721528887749j)
 a'1000'b'0001' : (-0.09666130691766739-0.13439182937145233j)
 a'1000'b'0010' : (-0.11500085145235062+0.49070459604263306j)
 a'1000'b'0100' : (-0.04762822762131691-0.06598378717899323j)
 a'1000'b'1000' : (-0.227739

### 2.A.4 Element access

Element access is modified from the original propsal to add more convient getter and setter functionalities.  The orginally proposed methods are availible with an addition function to add a value to an existing element.

In [29]:
wfFQE.set_ele(1, 8, .14159265359j)
wfFQE.add_ele(1, 8, 3.j)
imaginary_pi = wfFQE.get_ele(1, 8)
if fqeprint:
    print(-imaginary_pi*1.j)

(3.1415927410125732-0j)


The new components not originally in the proposal allow access of entire vectors for getting and setting methods.

In [30]:
interesting_states = wfn_from_cirq.get_coeff((2, 0), vec=[0,2])
if fqeprint:
    print(interesting_states)

[[-0.01153528+0.08957683j  0.        +0.j        ]
 [-0.06807666+0.16205184j  0.        +0.j        ]
 [-0.04735724+0.22589865j  0.        +0.j        ]
 [ 0.31179786-0.12040638j  0.        +0.j        ]]


In [31]:
interesting_states[:, 1] += 1. + .1j
interesting_states *= .7 - .3j
if fqeprint:
    print(interesting_states)

[[0.01879836+0.06616437j 0.73      -0.23000002j]
 [0.0009619 +0.13385929j 0.73      -0.23000002j]
 [0.03461954+0.17233622j 0.73      -0.23000002j]
 [0.18213658-0.17782383j 0.73      -0.23000002j]]


In [32]:
data = {}
data[(2, 0)] = interesting_states
wfn_from_cirq.set_wfn(vrange=[0, 2], strategy='from_data', raw_data=data)
if fqeprint:
    wfn_from_cirq.print_wfn()


Configuration nelectrons: 0 m_s: 0
Vector : 0
 a'00'b'00' : (0.0964515283703804+0.09214065223932266j)

Configuration nelectrons: 1 m_s: -1
Vector : 0
 a'00'b'01' : (-0.14026734232902527+0.28639745712280273j)
 a'00'b'10' : (0.16145852208137512-0.07011482119560242j)

Configuration nelectrons: 1 m_s: 1
Vector : 0
 a'01'b'00' : (0.0036136810667812824-0.05362788215279579j)
 a'10'b'00' : (0.08239555358886719+0.37468528747558594j)

Configuration nelectrons: 2 m_s: -2
Vector : 0
 a'00'b'11' : (-0.1846415400505066+0.22202734649181366j)

Configuration nelectrons: 2 m_s: 0
Vector : 0
 a'01'b'01' : (0.018798355013132095+0.06616436690092087j)
 a'01'b'10' : (0.0009618997573852539+0.1338592916727066j)
 a'10'b'01' : (0.0346195362508297+0.17233622074127197j)
 a'10'b'10' : (0.18213658034801483-0.17782382667064667j)

Configuration nelectrons: 2 m_s: 2
Vector : 0
 a'11'b'00' : (0.2513582110404968-0.2503160238265991j)

Configuration nelectrons: 3 m_s: -1
Vector : 0
 a'01'b'11' : (-0.14375174045562744-0.34

### 2.A.5 Printing/Saving

Printing is currently availible as alpha beta strings followed by the coefficient as well as orbital occupation representation.

In [33]:
print('String forrmatting')
wfn_from_cirq.print_wfn(fmt='str')
print('\n Occupation forrmatting')
wfn_from_cirq.print_wfn(fmt='occ')

String forrmatting

Configuration nelectrons: 0 m_s: 0
Vector : 0
 a'00'b'00' : (0.0964515283703804+0.09214065223932266j)

Configuration nelectrons: 1 m_s: -1
Vector : 0
 a'00'b'01' : (-0.14026734232902527+0.28639745712280273j)
 a'00'b'10' : (0.16145852208137512-0.07011482119560242j)

Configuration nelectrons: 1 m_s: 1
Vector : 0
 a'01'b'00' : (0.0036136810667812824-0.05362788215279579j)
 a'10'b'00' : (0.08239555358886719+0.37468528747558594j)

Configuration nelectrons: 2 m_s: -2
Vector : 0
 a'00'b'11' : (-0.1846415400505066+0.22202734649181366j)

Configuration nelectrons: 2 m_s: 0
Vector : 0
 a'01'b'01' : (0.018798355013132095+0.06616436690092087j)
 a'01'b'10' : (0.0009618997573852539+0.1338592916727066j)
 a'10'b'01' : (0.0346195362508297+0.17233622074127197j)
 a'10'b'10' : (0.18213658034801483-0.17782382667064667j)

Configuration nelectrons: 2 m_s: 2
Vector : 0
 a'11'b'00' : (0.2513582110404968-0.2503160238265991j)

Configuration nelectrons: 3 m_s: -1
Vector : 0
 a'01'b'11' : (-0.143

### 2.A.6 Fermionic algebra operations and their unitaries on the state

FermionOperators can be directly passed in to create a newwavefunction based on application of the operators.

In [34]:
from openfermion import FermionOperator
ops = FermionOperator('2^ 0', 1.2)
new_wfn = wfn_from_cirq.apply(ops)

In [35]:
new_wfn = fqe.apply(FermionOperator('0^ 0', 1.0), wfFQE)
if fqeprint:
    wfFQE.print_wfn()
    new_wfn.print_wfn()


Configuration nelectrons: 2 m_s: 0
Vector : 0
 a'0001'b'0001' : (-0.04786796122789383-0.11286675930023193j)
 a'0001'b'0010' : (-0.15004976093769073-0.0017647177446633577j)
 a'0001'b'0100' : (-0.08476771414279938-0.007398678921163082j)
 a'0001'b'1000' : 3.1415927410125732j
 a'0010'b'0001' : (0.08812709897756577+0.36353716254234314j)
 a'0010'b'0010' : (-0.05687916278839111-0.32680338621139526j)
 a'0010'b'0100' : (0.12052047997713089-0.08402305096387863j)
 a'0010'b'1000' : (0.14810921251773834+0.04006848484277725j)
 a'0100'b'0001' : (-0.22645647823810577+0.05994859337806702j)
 a'0100'b'0010' : (0.21680548787117004-0.1661015748977661j)
 a'0100'b'0100' : (0.0844455286860466-0.26342910528182983j)
 a'0100'b'1000' : (-0.1589016169309616+0.013544721528887749j)
 a'1000'b'0001' : (-0.09666130691766739-0.13439182937145233j)
 a'1000'b'0010' : (-0.11500085145235062+0.49070459604263306j)
 a'1000'b'0100' : (-0.04762822762131691-0.06598378717899323j)
 a'1000'b'1000' : (-0.2277396023273468-0.0497015826

The fqe has vdot functionality to get an inner product and provides a way to do completely general expectation values.

In [36]:
ops = FermionOperator('2^ 2', 0.5)
opexpec = fqe.vdot(wfFQE, wfFQE.apply(ops))
if fqeprint:
    print(opexpec)

(0.14754411578178406+0j)


In [37]:
ops = FermionOperator('2^ 2', 0.5)
opexpec = fqe.vdot(wfFQE, fqe.apply(ops, wfFQE))
if fqeprint:
    print(opexpec)

(0.14754411578178406+0j)


We can also do general unitary application of operators.  One slight modification to the proposal is that the operator passed in should Hermetian and not anti-Hermitian as stated before.

In [38]:
ops = FermionOperator('2^ 0', 0.2 - .3j) + FermionOperator('0^ 2', 0.2 + .3j)
wfFQE.print_wfn()
wfFQE2 = wfFQE.apply_generated_unitary(ops, algo='taylor')
wfFQE2.print_wfn()
ops = FermionOperator('2^ 0', 0.2 - .3j) + FermionOperator('0^ 2', 0.2 + .3j)
wfFQE2 = fqe.apply_generated_unitary(ops, wfFQE, algo='taylor')
wfFQE2.print_wfn()


Configuration nelectrons: 2 m_s: 0
Vector : 0
 a'0001'b'0001' : (-0.04786796122789383-0.11286675930023193j)
 a'0001'b'0010' : (-0.15004976093769073-0.0017647177446633577j)
 a'0001'b'0100' : (-0.08476771414279938-0.007398678921163082j)
 a'0001'b'1000' : 3.1415927410125732j
 a'0010'b'0001' : (0.08812709897756577+0.36353716254234314j)
 a'0010'b'0010' : (-0.05687916278839111-0.32680338621139526j)
 a'0010'b'0100' : (0.12052047997713089-0.08402305096387863j)
 a'0010'b'1000' : (0.14810921251773834+0.04006848484277725j)
 a'0100'b'0001' : (-0.22645647823810577+0.05994859337806702j)
 a'0100'b'0010' : (0.21680548787117004-0.1661015748977661j)
 a'0100'b'0100' : (0.0844455286860466-0.26342910528182983j)
 a'0100'b'1000' : (-0.1589016169309616+0.013544721528887749j)
 a'1000'b'0001' : (-0.09666130691766739-0.13439182937145233j)
 a'1000'b'0010' : (-0.11500085145235062+0.49070459604263306j)
 a'1000'b'0100' : (-0.04762822762131691-0.06598378717899323j)
 a'1000'b'1000' : (-0.2277396023273468-0.0497015826

### 3.B.1 Hamiltonian data structure

Hamiltonians can be incorporated from OpenFermion objects or initialized from numpy arrays.  Notice a slight change in the arguments to get_two_body to include a constant potential term and additionally a chemical potential to be consistent with OpenFermion.

In [39]:
from openfermion import get_quadratic_hamiltonian
ops = FermionOperator((), 1.) + FermionOperator('1^ 1', 3.) + FermionOperator('1^ 2', 3. + 4.j) + FermionOperator('2^ 1', 3. - 4.j) + FermionOperator('3^ 4', 2. + 5.j) + FermionOperator('4^ 3', 2. - 5.j)
hamOF = get_quadratic_hamiltonian(ops, -0.03)
hamFQE = fqe.get_hamiltonian_from_openfermion(hamOF)
hamFQE = fqe.get_quadratic_hamiltonian(ops, -0.03)

Symmetry is passed in by lists containing permutation operations, a phase and whether or not complex conjugation is used.  Below are examples of a Hermetian matrix and a real anti-symmetric matrix.

In [40]:
e = 1.0
t = numpy.random.rand(4, 4) + numpy.random.rand(4, 4)*1.j
h = t + numpy.conjugate(t.T)
symmh = [
    [[1, 2], 1.0, False],
    [[2, 1], 1.0, True]
]
t = numpy.random.rand(4, 4, 4, 4)
g = t - t.T
symmg = [
    [[1, 2, 3, 4], 1.0, True],
    [[4, 3, 2, 1], -1.0, True]
]
hamFQE = fqe.get_two_body_hamiltonian(-0.3, h, g, 0.0, symmh, symmg)

The FQE will also return PolynomialTensor objects for parsing by OpenFermion objects.

In [41]:
hamOF = fqe.hamiltonian_to_openfermion(hamFQE)

### Saving and Loading Wavefunctions

Saving or loading a wavefunction is accomplished by passing a file name to the API and optionally a path to that filename.  If no path is given, the current working directory will be used.  If a path is used, the file will be searched for as

path + '/' + filename

In [44]:
wfnFqe = fqe.Wavefunction([[2, 2, 2], [2, 0, 2], [2, -2, 2]])
wfnFqe.set_wfn(strategy='random')
wfnFqe.print_wfn()


Configuration nelectrons: 2 m_s: -2
Vector : 0
 a'00'b'11' : (-0.6559730172157288+0.754784345626831j)

Configuration nelectrons: 2 m_s: 0
Vector : 0
 a'01'b'01' : (0.4016709625720978+0.49356693029403687j)
 a'01'b'10' : (-0.010087961331009865+0.5587083697319031j)
 a'10'b'01' : (-0.006604856811463833-0.3804541230201721j)
 a'10'b'10' : (0.2994023859500885+0.21991939842700958j)

Configuration nelectrons: 2 m_s: 2
Vector : 0
 a'11'b'00' : (0.5662325024604797-0.8242455720901489j)


In [50]:
wfnFqe.save('random_wavefunction')

Once it is saved you can return to the wavefunction by creating new object and then 'read' the data into it with the same syntax.

In [52]:
progress = fqe.Wavefunction()
progress.read('random_wavefunction')
progress.print_wfn()


Configuration nelectrons: 2 m_s: -2
Vector : 0
 a'00'b'11' : (-0.6559730172157288+0.754784345626831j)

Configuration nelectrons: 2 m_s: 0
Vector : 0
 a'01'b'01' : (0.4016709625720978+0.49356693029403687j)
 a'01'b'10' : (-0.010087961331009865+0.5587083697319031j)
 a'10'b'01' : (-0.006604856811463833-0.3804541230201721j)
 a'10'b'10' : (0.2994023859500885+0.21991939842700958j)

Configuration nelectrons: 2 m_s: 2
Vector : 0
 a'11'b'00' : (0.5662325024604797-0.8242455720901489j)
